In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
from contextlib import asynccontextmanager
from azure.storage.blob.aio import BlobServiceClient
from kv.azure.blob import BlobKV, BlobContainerKV
CONN_STR = os.environ['BLOB_CONN_STR']

In [2]:
kv = BlobContainerKV(client=lambda: BlobServiceClient.from_connection_string(CONN_STR), container='test')

In [4]:
await kv.insert('hello', b'world')

Right(value=None, tag='right')

In [5]:
await kv.keys()

Right(value=['hello'], tag='right')